In [2]:
import pandas as pd

In [3]:
player_info_df = pd.read_csv('Players_Info_2024.csv')
player_info_df.drop(['Player Nationality', 'Date of Birth', 'IPL Debut', 'About', 'Batting Style', 'Bowling Style', 'Player Salary'], axis=1, inplace=True)
player_info_df.head()

,Player Name,Team Name,Player Role
0,MS Dhoni,CSK,Wicketkeeper Batter
1,Devon Conway,CSK,Wicketkeeper Batter
2,Ruturaj Gaikwad,CSK,Batter
3,Ajinkya Rahane,CSK,Top order Batter
4,Shaik Rasheed,CSK,Batter


In [4]:
batting_df = pd.read_json('batting-2024.json')
batting_df = batting_df[['StrikerName', 'ClientPlayerID']]
batting_df = batting_df.rename({'StrikerName':'Player Name'}, axis=1)
batting_df.head()

,Player Name,ClientPlayerID
0,Virat Kohli,164
1,Riyan Parag,4445
2,Sanju Samson,258
3,Heinrich Klaasen,3869
4,Shubman Gill,3761


In [52]:
bowling_df = pd.read_json('bowling-2023.json')
bowling_df = bowling_df[['BowlerName', 'ClientPlayerID']]
bowling_df = bowling_df.rename({'BowlerName':'Player Name'}, axis=1)
bowling_df.head()

,Player Name,ClientPlayerID
0,Mohammad Shami,94
1,Mohit Sharma,1107
2,Rashid Khan,2885
3,Piyush Chawla,76
4,Yuzvendra Chahal,111


In [75]:
combined_players = pd.concat([bowling_df, batting_df])
combined_players.head()

,Player Name,ClientPlayerID
0,Mohammad Shami,94
1,Mohit Sharma,1107
2,Rashid Khan,2885
3,Piyush Chawla,76
4,Yuzvendra Chahal,111


In [76]:
len(bowling_df) + len(batting_df) == len(combined_players)

True

In [77]:
combined_players['ClientPlayerID'] = combined_players['ClientPlayerID'].apply(lambda x: int(x) if x != '' else -1)

In [78]:
combined_players = combined_players.drop_duplicates(subset=['ClientPlayerID'])

In [65]:
combined_players[combined_players['Player Name'] == 'Ravindra Jadeja']

,Player Name,ClientPlayerID
6,Ravindra Jadeja,9


In [79]:
print(len(combined_players), len(player_info_df))

197 245


In [68]:
# combined_players.merge(player_info_df, on=['Player Name'])

,Player Name,ClientPlayerID,Team Name,Player Role
0,Mohit Sharma,1107,GT,Bowler
1,Rashid Khan,2885,GT,Bowling Allrounder
2,Piyush Chawla,76,MI,Bowling Allrounder
3,Yuzvendra Chahal,111,RR,Bowler
4,Tushar Deshpande,3257,CSK,Bowler
...,...,...,...,...
141,Abishek Porel,33333,DC,Wicketkeeper Batter
142,Yash Dhull,20623,DC,Top order Batter
143,Sanvir Singh,5616,SRH,Allrounder
144,Jaydev Unadkat,86,SRH,Bowler


In [99]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Function to find best match for each name in df1
def find_best_match(name, choices):
    best_match, score = process.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    if score >= 83:  # Adjust the threshold according to your preference
        return best_match, score
    else:
        return None, 0

# Create a mapping between names in df1 and df2
name_mapping = {name: find_best_match(name, list(player_info_df['Player Name']))[0] for name in list(combined_players['Player Name'])}
score_mapping = {name: find_best_match(name, list(player_info_df['Player Name']))[1] for name in list(combined_players['Player Name'])}

# Map names from combined_players to corresponding names in player_info_df
combined_players['Mapped Name'] = combined_players['Player Name'].map(name_mapping)
combined_players['Mapped Score'] = combined_players['Player Name'].map(score_mapping)

# Merge dataframes using the mapped names
merged_df = pd.merge(combined_players, player_info_df, left_on='Mapped Name', right_on='Player Name', how='inner')

In [100]:
merged_df[merged_df['Player Name_x'] != merged_df['Mapped Name']]

,Player Name_x,ClientPlayerID,Mapped Name,Mapped Score,Player Name_y,Team Name,Player Role
0,Mohammad Shami,94,Mohammed Shami,93,Mohammed Shami,GT,Bowler
25,Naveen-Ul-Haq,5681,Naveen-ul-Haq,100,Naveen-ul-Haq,LSG,Bowler
46,Josh Little,5683,Joshua Little,92,Joshua Little,GT,Bowler
54,Kumar Kartikeya Singh,20629,Kumar Kartikeya,83,Kumar Kartikeya,MI,Bowler
77,Kuldip Yadav,12480,Kuldeep Yadav,88,Kuldeep Yadav,DC,Bowler
81,Praveen Dubey,3188,Pravin Dubey,88,Pravin Dubey,DC,Bowler
97,Faf Du Plessis,24,Faf du Plessis,100,Faf du Plessis,RCB,Middle order Batter
114,Sai Sudharsan,20592,B. Sai Sudharsan,93,B. Sai Sudharsan,GT,Top order Batter
116,Simran Singh,5436,Prabhsimran Singh,83,Prabhsimran Singh,PBKS,Wicketkeeper Batter
123,K L Rahul,1125,KL Rahul,94,KL Rahul,LSG,Wicketkeeper Batter


In [101]:
merged_df.to_json('client_id_player_role_team_map.json', orient='records')